In [74]:
from os import listdir, getcwd, rename, makedirs, remove
from os.path import isfile, join, isdir, exists
import re
import numpy as np
import pandas as pd
from tqdm import tqdm
import json
import requests

### Common functions

In [75]:
def get_dirs(path):
    return [f for f in listdir(path) if isdir(join(path, f))]

def get_files(path):
    return [join(path, f) for f in listdir(path) if isfile(join(path, f)) and f.endswith(".tsv")]

def read_metadata_without_fields(path):
    metadata = pd.read_csv(path, sep='\t', na_values="", low_memory=False)
    if 'Field' in metadata.columns:
        metadata = metadata.loc[metadata.Field.str.startswith('#') != True,].reset_index(drop=True)
        metadata = metadata.drop('Field', axis=1)
    return metadata

def read_metadata_with_fields(path):
    metadata = pd.read_csv(path, sep='\t', na_values="", low_memory=False)
    return metadata

def sort_case_insensitive(sort_list):
    return sorted(sort_list, key=str.casefold)


In [76]:
start_dir = getcwd()
home = "/Users/tushar/CancerModels/pdxfinder-data/data/UPDOG/"
providers = sorted(get_dirs(home))
API_scores = "https://www.cancermodels.org/api/search_index?data_source=eq."
API_select = "&select=pdcm_model_id,external_model_id,patient_sample_id,data_source,scores,model_type,dataset_available"

In [77]:
def fetch(url):
    response = requests.get(url)
    # Check if the API request was successful (status code 200)
    if response.status_code != 200:
        return None
    # Flatten the JSON data
    data = json.loads(response.text)
    flattened_data = pd.json_normalize(data)
    # Create a DataFrame from the flattened JSON data
    df = pd.DataFrame(flattened_data)
    return df

def group_df(df):
    # Group by "model_type" and calculate counts, min, and max scores
    grouped = df.groupby("model_type").agg(
        data_source=pd.NamedAgg(column="data_source", aggfunc=lambda x: x.value_counts().idxmax()),
        model_count=pd.NamedAgg(column="model_type", aggfunc="count"),
        min_data_score=pd.NamedAgg(column="scores.data_score", aggfunc="min"),
        max_data_score=pd.NamedAgg(column="scores.data_score", aggfunc="max"),
        min_pdx_metadata_score=pd.NamedAgg(column="scores.pdx_metadata_score", aggfunc="min"),
        max_pdx_metadata_score=pd.NamedAgg(column="scores.pdx_metadata_score", aggfunc="max"),
        dataset_available=pd.NamedAgg(column="dataset_available", aggfunc=lambda x: x.value_counts().idxmax())
    )
    # Reset the index for a cleaner DataFrame
    grouped = grouped.reset_index()
    return grouped.sort_values(by=["max_data_score", "max_pdx_metadata_score"], ascending=[False, False])

In [78]:
def assess(api_url):
    assessment = fetch(api_url).fillna("")
    if assessment is None:
        return None
    return group_df(assessment)

def get_dataset_assessment(providers, scores, select):
    da = pd.DataFrame()
    for i in tqdm(range(0, len(providers)), desc ="Generating data assessment report: "): ## get_dirs will get the provider dirs in updog
        provider = providers[i]
        url = scores+provider+select
        da = pd.concat([da, assess(url)], ignore_index=True)
    return da

In [79]:
assessment = get_dataset_assessment(providers, API_scores, API_select)
assessment

Generating data assessment report: 100%|██████████| 34/34 [00:06<00:00,  5.33it/s]


,model_type,data_source,model_count,min_data_score,max_data_score,min_pdx_metadata_score,max_pdx_metadata_score,dataset_available
0,organoid,BROD,23,0,14,0,0,
1,other,BROD,65,0,14,0,0,[mutation]
2,PDX,CCIA,90,28,57,60,71,"[mutation, copy number alteration, expression,..."
3,PDX,CHOP,34,57,57,66,86,"[mutation, copy number alteration, expression,..."
4,cell line,CMP,1500,0,42,0,0,"[mutation, copy number alteration, expression]"
5,organoid,CMP,41,0,42,0,0,"[mutation, copy number alteration, expression]"
6,PDX,CRL,540,0,71,57,90,"[mutation, copy number alteration, expression]"
7,organoid,CSHL,53,0,14,0,0,[mutation]
8,PDX,Curie-BC,34,28,57,83,89,"[mutation, cytogenetics, publication]"
9,organoid,Curie-BC,15,0,14,0,0,
